In [1]:
import numpy
import matplotlib.pyplot as plt
import pandas
import os
import sys
import scipy.linalg
import scipy.stats
import importlib
import pickle

In [2]:
# # read in posterior
# posterior = numpy.loadtxt ("/Users/steinrue/googleDrive/misc/data_matthias/posterior0.csv")
# # load some files maybe
# # recombination map (in morgan)
# recoMap = numpy.loadtxt ("/Users/steinrue/googleDrive/misc/data_matthias/map.csv", delimiter=',')
# print (recoMap.shape)
# # readcounts for target individual
# # first line: reads for ref; second line: reads for alt
# target = numpy.loadtxt ("/Users/steinrue/googleDrive/misc/data_matthias/readcounts.csv", delimiter=',')
# print (target.shape)
# numSnps = target.shape[1]
# # read in references
# # 0 in refHaps is refAllele
# refHaps = numpy.loadtxt ("/Users/steinrue/googleDrive/misc/data_matthias/refs.csv", delimiter=',')
# print (refHaps.shape)

In [3]:
# # make it small for now, so we see what we are doing
# numHaps = 8
# numLoci = 12
# refHaps = refHaps[0:numHaps,0:numLoci]
# target = target[:,0:numLoci]
# recoMap = recoMap[0:numLoci]

# # refHaps = refHaps[360:380,:]

In [4]:
# pickle_rick = [refHaps, target, recoMap]
# pickle.dump (pickle_rick, open("tmp.pickle", "wb"))

In [5]:
pickle_rick = pickle.load (open("tmp.pickle", "rb"))
myRefHaps = pickle_rick[0]
myTarget = pickle_rick[1]
myRecoMap = pickle_rick[2]

In [6]:
# make it small for now, so we see what we are doing
numHaps = 8
numLoci = 3
myRefHaps = myRefHaps[0:numHaps,0:numLoci]
myTarget = myTarget[:,0:numLoci]
myRecoMap = myRecoMap[0:numLoci]

In [7]:
# get the path right, for some reason cython makes trouble
sys.path.append("../../Python3/")
sys.path.append("../../Python3/Python3/")
sys.path.append("../../PackagesSupport/loadEigenstrat/")

In [8]:
from hmm_inference import HMM_Analyze
import msTransitions
import msEmissions
import msFun

In [9]:
def fake_transition_matrices (rates, rec_v):
    transition_matrices = numpy.zeros((len(rec_v),6,6))
    for i in range(len(rec_v)):
        transition_matrices[i] = numpy.eye(6)
#         transition_matrices[i] = numpy.zeros((6,6))
#         transition_matrices[i,:2,:2] = 0.5
#         transition_matrices[i,2:4,2:4] = 0.5
#         transition_matrices[i,2:4,2:4] = numpy.ones((2,2)) - numpy.eye(2)
#         transition_matrices[i,4:6,4:6] = 0.5
    return transition_matrices

In [10]:
importlib.reload(sys.modules["msTransitions"])
from msTransitions import rate_matrix_oneSL, exponentiate_rate_matrices, allele_frequencies
from msTransitions import one_step_transitions_reference, non_roh_state_map, non_roh_transition_matrices
from msHMM import fancy_initial_distribution, stationary_inital_distribution
importlib.reload(sys.modules["msEmissions"])
from msEmissions import extended_genotype_emissions, extended_rc_emission_log, extended_binom_emission
importlib.reload(sys.modules["msFun"])
from msFun import extended_fwd_bkwd_fast


def analyze_targets (targets, refHaps, recoMap, pi_s = 1e-4, pi_l = 1e-4,
                     in_S = 100, in_L = 2, out_S = 400, out_L = 10, roh_jump = 200,
                     e_rate_ref=1e-3, e_rate=1e-2):
    
    # see how many targets we have
    if (len(targets.shape) == 2):
        # only one target, prepare accordingly
        targets = targets[None,:,:]
    # if already multiple targets, all fine
    
    # prepare target independent things
    
    # set up all the necessary stuff
    n_ref = refHaps.shape[0]
    n_loci = refHaps.shape[1]

    # prepare the recombination map
    # fake using a member function of class HMM_Analyze
    blub = type('', (), {})()
    blub.r_map = recoMap
    blub.output = False
    r_map = HMM_Analyze.prepare_rmap (self=blub)

    # transition rate matrix Q
    # t_mat = new_calc_transitions (roh_in=100, roh_out=100, roh_jump=100)
    Q = rate_matrix_oneSL (in_S = in_S, in_L = in_L, out_S = out_S, out_L = out_L, roh_jump = roh_jump)
#     Q = numpy.zeros((6,6))
    # marginal properbilities
    f_marg = allele_frequencies (refHaps)

    # initial distribution
#     init_d = numpy.log (fancy_initial_distribution (n_ref, f_marg[0], pi_s=pi_s, pi_l=pi_l))
    init_d = numpy.log (stationary_inital_distribution (n_ref, f_marg[0], Q))
    
    # transition
    transition_matrices = exponentiate_rate_matrices (rates=Q, rec_v=r_map)
#     transition_matrices = fake_transition_matrices (rates=Q, rec_v=r_map)
    # also need the allele frequencies
    f_trans = one_step_transitions_reference (refHaps, f_marg)
    # get transition matrices between the non-ROH states
    non_roh_transitions = non_roh_transition_matrices (transition_matrices, f_trans, f_marg)

    # emission
    # prepare the emissions for all loci
    e_mat_geno = extended_genotype_emissions (refHaps, e_rate_ref)
    
    # now the target specific things
    all_ll = numpy.zeros ((targets.shape[0],targets.shape[2]))
    for i in range(targets.shape[0]):
        thisTarget = targets[i]
#         print (thisTarget)
        
        # target specific emissions
        e_mat_full = numpy.log (extended_binom_emission (ob_stat=thisTarget, e_mat=e_mat_geno, e_rate=e_rate))
        # just some checks
        assert(numpy.max(e_mat_full) < 0) 

        # I think we don't need these for now
        # n_states = np.shape(e_mat)[0]
        # n_loci = np.shape(e_mat)[1]

        (newPost, fwd, bwd, tot_ll) = extended_fwd_bkwd_fast (init_d=init_d, e_prob0=e_mat_full,
                                          t=transition_matrices, f_marg=f_marg, f_trans=f_trans,
                                          non_roh_transitions=non_roh_transitions, full=True)
        # newPost = old_fwd_bkwd_fast (e_prob0=e_mat, t=t_mat_full, in_val=in_val)
        # (post, fwd, bwd, tot_ll) = new_fwd_bkwd_fast (e_mat, t_mat=None, t_mat_full, in_val, full=True)
        # sum over hidden states
        all_ll[i,:] = numpy.sum(numpy.exp(fwd),axis=1)
        print (numpy.sum (numpy.exp(newPost), axis=1))
    
    # give it away now
    return all_ll

In [11]:
results = analyze_targets (myTarget, myRefHaps, myRecoMap)

Memory Usage Full:
Memory Usage: 137.777152 mB
[1. 1. 1.]


In [12]:
allTargets = -numpy.ones((12,2,3))

allTargets[0] = numpy.array([[0,0,0],
                             [1,1,2]])

allTargets[1] = numpy.array([[0,0,1],
                             [1,1,1]])

allTargets[2] = numpy.array([[0,0,2],
                             [1,1,0]])

allTargets[3] = numpy.array([[0,1,0],
                             [1,0,2]])

allTargets[4] = numpy.array([[0,1,1],
                             [1,0,1]])

allTargets[5] = numpy.array([[0,1,2],
                             [1,0,0]])

allTargets[6] = numpy.array([[1,0,0],
                             [0,1,2]])

allTargets[7] = numpy.array([[1,0,1],
                             [0,1,1]])

allTargets[8] = numpy.array([[1,0,2],
                             [0,1,0]])

allTargets[9] = numpy.array([[1,1,0],
                             [0,0,2]])

allTargets[10] = numpy.array([[1,1,1],
                              [0,0,1]])

allTargets[11] = numpy.array([[1,1,2],
                              [0,0,0]])


In [13]:
# results = analyze_targets (allTargets, myRefHaps, myRecoMap, pi_s = 0, pi_l = 0)
results = analyze_targets (allTargets, myRefHaps, myRecoMap)

Memory Usage Full:
Memory Usage: 137.834496 mB
[1. 1. 1.]
Memory Usage Full:
Memory Usage: 137.834496 mB
[1. 1. 1.]
Memory Usage Full:
Memory Usage: 137.834496 mB
[1. 1. 1.]
Memory Usage Full:
Memory Usage: 137.834496 mB
[1. 1. 1.]
Memory Usage Full:
Memory Usage: 137.838592 mB
[1. 1. 1.]
Memory Usage Full:
Memory Usage: 137.838592 mB
[1. 1. 1.]
Memory Usage Full:
Memory Usage: 137.838592 mB
[1. 1. 1.]
Memory Usage Full:
Memory Usage: 137.838592 mB
[1. 1. 1.]
Memory Usage Full:
Memory Usage: 137.838592 mB
[1. 1. 1.]
Memory Usage Full:
Memory Usage: 137.838592 mB
[1. 1. 1.]
Memory Usage Full:
Memory Usage: 137.838592 mB
[1. 1. 1.]
Memory Usage Full:
Memory Usage: 137.838592 mB
[1. 1. 1.]


In [14]:
numpy.sum(results[:,0])/6

0.9999999999999999

In [15]:
numpy.sum(results[:,1])/3

1.0

In [16]:
numpy.sum(results[:,2])

0.9999999999999998